<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Preprocess Data
</div>

In [99]:
import pandas as pd

### Read raw data that we have collected

In [100]:
raw_df = pd.read_csv("../data/raw/raw_data.csv")
raw_df.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1961,ha,1000.0,E,Estimated value,NaN
1,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1961,100 g/ha,7000.0,E,Estimated value,NaN
2,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1961,t,700.0,E,Estimated value,NaN
3,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,1962,ha,1000.0,E,Estimated value,NaN
4,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,1962,100 g/ha,7000.0,E,Estimated value,NaN


-   Look like the `Note` column is all NaN, let check it:
    -   How many kind of data in the column?
    -   The ratio between data with no values and these other?

In [101]:
# check the variety of value
print("Unique values in the column:",raw_df['Note'].unique())

# check the ratio between loss and other data
raw_df['Note'].count()

Unique values in the column: [nan 'Unofficial figure']


548

-   Look like it's contain some useful information, so we keep it

### Meaning of each columns and rows

// Explain here

In [102]:
#TODO: Use code to explain if need

### Clear duplicates columns:
-   There is `Year Code` and `Year` columns, they have the same values, so we drop the `Year Code`.

In [103]:
raw_df = raw_df.drop(columns=['Year Code'])
raw_df.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,ha,1000.0,E,Estimated value,NaN
1,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,100 g/ha,7000.0,E,Estimated value,NaN
2,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,t,700.0,E,Estimated value,NaN
3,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,ha,1000.0,E,Estimated value,NaN
4,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,100 g/ha,7000.0,E,Estimated value,NaN


### Check for duplicate rows

In [104]:
index = raw_df.index
detectDupSeries = index.duplicated(keep='first')
num_duplicated_rows = detectDupSeries.sum()

In [105]:
if num_duplicated_rows == 0:
    print(f"Your raw data have no duplicated line.!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"Your raw data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate your raw data.!")

Your raw data have no duplicated line.!


### Check for the distribution of each value associated with an item.

In [106]:
def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def median(df):
    return (df.quantile(0.5)).round(1)

def lower_quartile(df):
    return (df.quantile(0.25)).round(1)

def upper_quartile(df):
    return (df.quantile(0.75)).round(1)

In [107]:
# make a DataFrame to store each item info about how their data distribute
data_distribute_df = pd.DataFrame()

for item in raw_df['Item'].unique():
    item_df = raw_df[raw_df['Item'] == item] # get rows that have the same item

    num_col_info_df = item_df.select_dtypes(exclude='object')
    num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])

    num_col_info_df = num_col_info_df.transpose()

    num_col_info_df['Item'] = item
    
    data_distribute_df = pd.concat([data_distribute_df, num_col_info_df])

In [108]:
data_distribute_df

,missing_ratio,min,lower_quartile,median,upper_quartile,max,Item
Area Code,0.0,237.0,237.0,237.0,237.0,237.00,"Anise, badian, coriander, cumin, caraway, fenn..."
Element Code,0.0,5312.0,5312.0,5419.0,5510.0,5510.00,"Anise, badian, coriander, cumin, caraway, fenn..."
Item Code,0.0,711.0,711.0,711.0,711.0,711.00,"Anise, badian, coriander, cumin, caraway, fenn..."
Year,0.0,1961.0,1976.0,1991.0,2006.0,2021.00,"Anise, badian, coriander, cumin, caraway, fenn..."
Value,0.0,300.0,1000.0,4200.0,6885.5,25817.00,"Anise, badian, coriander, cumin, caraway, fenn..."
...,...,...,...,...,...,...,...
Area Code,0.0,237.0,237.0,237.0,237.0,237.00,Vegetables Primary
Element Code,0.0,5312.0,5312.0,5419.0,5510.0,5510.00,Vegetables Primary
Item Code,0.0,1735.0,1735.0,1735.0,1735.0,1735.00,Vegetables Primary
Year,0.0,1961.0,1976.0,1991.0,2006.0,2021.00,Vegetables Primary


### Determine the features that have a large number of missing values

In [109]:
percent_missing = raw_df.isnull().sum() * 100 / len(raw_df)
missing_value_df = pd.DataFrame({'column_name': raw_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Domain Code,Domain Code,0.000000
Domain,Domain,0.000000
Area Code,Area Code,0.000000
Area,Area,0.000000
Element Code,Element Code,0.000000
Element,Element,0.000000
Item Code,Item Code,0.000000
Item,Item,0.000000
Year,Year,0.000000
Unit,Unit,0.000000


-   We gonna choose >= 75% is missing a lot of values, so we have to drop the `Note` column (~96.364%).

In [110]:
raw_df = raw_df.drop(columns='Note')
raw_df

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Value,Flag,Flag Description
0,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,ha,1000.00,E,Estimated value
1,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,100 g/ha,7000.00,E,Estimated value
2,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,t,700.00,E,Estimated value
3,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,ha,1000.00,E,Estimated value
4,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,100 g/ha,7000.00,E,Estimated value
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15065,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,1735,Vegetables Primary,2020,100 g/ha,170584.00,E,Estimated value
15066,QCL,Crops and livestock products,237,Viet Nam,5510,Production,1735,Vegetables Primary,2020,t,17006433.11,E,Estimated value
15067,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,1735,Vegetables Primary,2021,ha,1002974.00,E,Estimated value
15068,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,1735,Vegetables Primary,2021,100 g/ha,171731.00,E,Estimated value


### Check data type of each columns

In [111]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15070 entries, 0 to 15069
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       15070 non-null  object 
 1   Domain            15070 non-null  object 
 2   Area Code         15070 non-null  int64  
 3   Area              15070 non-null  object 
 4   Element Code      15070 non-null  int64  
 5   Element           15070 non-null  object 
 6   Item Code         15070 non-null  int64  
 7   Item              15070 non-null  object 
 8   Year              15070 non-null  int64  
 9   Unit              15070 non-null  object 
 10  Value             15070 non-null  float64
 11  Flag              15070 non-null  object 
 12  Flag Description  15070 non-null  object 
dtypes: float64(1), int64(4), object(8)
memory usage: 1.5+ MB


`Area Code`, `Element Code`, `Item Code`, `Year` have numeric type. However, their magnitude does not have a significance. And, `Year` actually represents a period instead of a number, `Area Code`, `Element Code`, `Item Code` is just a category. Thus, they can be convert to categorical type.

In [112]:
for col in raw_df.drop(columns= 'Value').columns:
    raw_df[col] = raw_df[col].astype(str)
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15070 entries, 0 to 15069
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       15070 non-null  object 
 1   Domain            15070 non-null  object 
 2   Area Code         15070 non-null  object 
 3   Area              15070 non-null  object 
 4   Element Code      15070 non-null  object 
 5   Element           15070 non-null  object 
 6   Item Code         15070 non-null  object 
 7   Item              15070 non-null  object 
 8   Year              15070 non-null  object 
 9   Unit              15070 non-null  object 
 10  Value             15070 non-null  float64
 11  Flag              15070 non-null  object 
 12  Flag Description  15070 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.5+ MB


### Observation about the distribution of data
-   

### We try to merge data of same year into 1 row

In [113]:
raw_df

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Value,Flag,Flag Description
0,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,ha,1000.00,E,Estimated value
1,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,100 g/ha,7000.00,E,Estimated value
2,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,t,700.00,E,Estimated value
3,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,ha,1000.00,E,Estimated value
4,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,100 g/ha,7000.00,E,Estimated value
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15065,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,1735,Vegetables Primary,2020,100 g/ha,170584.00,E,Estimated value
15066,QCL,Crops and livestock products,237,Viet Nam,5510,Production,1735,Vegetables Primary,2020,t,17006433.11,E,Estimated value
15067,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,1735,Vegetables Primary,2021,ha,1002974.00,E,Estimated value
15068,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,1735,Vegetables Primary,2021,100 g/ha,171731.00,E,Estimated value


-   Split the data to 3 dataframes, each for one element

In [114]:
area_harvest_df = raw_df[raw_df['Element'] == 'Area harvested']
yield_df = raw_df[raw_df['Element'] == 'Yield']
production_df = raw_df[raw_df['Element'] == 'Production']

display(area_harvest_df.head(3))
display(yield_df.head(3))
display(production_df.head(3))

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Value,Flag,Flag Description
0,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,ha,1000.0,E,Estimated value
3,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,ha,1000.0,E,Estimated value
6,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,ha,1000.0,E,Estimated value


,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Value,Flag,Flag Description
1,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,100 g/ha,7000.0,E,Estimated value
4,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,100 g/ha,7000.0,E,Estimated value
7,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,100 g/ha,7000.0,E,Estimated value


,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Value,Flag,Flag Description
2,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,t,700.0,E,Estimated value
5,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,t,700.0,E,Estimated value
8,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,t,700.0,E,Estimated value


-   Change the name of the column `Value` to `'{Element} ({Unit})'`

In [115]:
element = area_harvest_df['Element'].values[0]
unit = area_harvest_df['Unit'].values[0]
area_harvest_df = area_harvest_df.rename(columns={'Value' : f"{element} ({unit})"})

element = yield_df['Element'].values[0]
unit = yield_df['Unit'].values[0]
yield_df = yield_df.rename(columns={'Value' : f"{element} ({unit})"})

element = production_df['Element'].values[0]
unit = production_df['Unit'].values[0]
production_df = production_df.rename(columns={'Value' : f"{element} ({unit})"})

In [116]:
# Let check
display(area_harvest_df.head(3))
display(yield_df.head(3))
display(production_df.head(3))

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Area harvested (ha),Flag,Flag Description
0,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,ha,1000.0,E,Estimated value
3,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,ha,1000.0,E,Estimated value
6,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,ha,1000.0,E,Estimated value


,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Yield (100 g/ha),Flag,Flag Description
1,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,100 g/ha,7000.0,E,Estimated value
4,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,100 g/ha,7000.0,E,Estimated value
7,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,100 g/ha,7000.0,E,Estimated value


,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Production (t),Flag,Flag Description
2,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,t,700.0,E,Estimated value
5,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,t,700.0,E,Estimated value
8,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,t,700.0,E,Estimated value


-   Now we drop the `Element`, `Element Code` and `Unit` columns
-   Also have to drop the `Flag` and `Flag Description` because with confict with the merge

In [117]:
area_harvest_df = area_harvest_df.drop(columns=['Element', 'Element Code', 'Unit', 'Flag', 'Flag Description'])

yield_df = yield_df.drop(columns=['Element', 'Element Code', 'Unit', 'Flag', 'Flag Description'])

production_df = production_df.drop(columns=['Element', 'Element Code', 'Unit', 'Flag', 'Flag Description'])

In [118]:
# Let check
display(area_harvest_df.head(3))
display(yield_df.head(3))
display(production_df.head(3))

,Domain Code,Domain,Area Code,Area,Item Code,Item,Year,Area harvested (ha)
0,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1000.0
3,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,1000.0
6,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,1000.0


,Domain Code,Domain,Area Code,Area,Item Code,Item,Year,Yield (100 g/ha)
1,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,7000.0
4,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,7000.0
7,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,7000.0


,Domain Code,Domain,Area Code,Area,Item Code,Item,Year,Production (t)
2,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,700.0
5,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,700.0
8,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,700.0


-   Let merge them

In [123]:
raw_df = pd.merge(area_harvest_df, yield_df, on=['Year', 'Item Code', 'Area', 'Area Code', 'Domain', 'Domain Code', 'Item'])
raw_df = pd.merge(raw_df, production_df, on=['Year', 'Item Code', 'Area', 'Area Code', 'Domain', 'Domain Code', 'Item'])
raw_df

,Domain Code,Domain,Area Code,Area,Item Code,Item,Year,Area harvested (ha),Yield (100 g/ha),Production (t)
0,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1000.0,7000.0,700.00
1,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,1000.0,7000.0,700.00
2,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1963,1000.0,7000.0,700.00
3,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1964,1000.0,8500.0,850.00
4,QCL,Crops and livestock products,237,Viet Nam,711,"Anise, badian, coriander, cumin, caraway, fenn...",1965,1000.0,8700.0,870.00
...,...,...,...,...,...,...,...,...,...,...
2978,QCL,Crops and livestock products,237,Viet Nam,1735,Vegetables Primary,2017,984965.0,159639.0,15723915.48
2979,QCL,Crops and livestock products,237,Viet Nam,1735,Vegetables Primary,2018,1006678.0,161589.0,16266791.68
2980,QCL,Crops and livestock products,237,Viet Nam,1735,Vegetables Primary,2019,991058.0,168246.0,16674164.74
2981,QCL,Crops and livestock products,237,Viet Nam,1735,Vegetables Primary,2020,996952.0,170584.0,17006433.11


### Save processed data to a new csv file

In [122]:
raw_df.to_csv("../data/processed/processed_data.csv", index= False)